
# We seek to build a generic architecture to model an epithelium.

<hr/>

## Documentation

### Boost

* [Official boost.python  documentation](http://www.boost.org/doc/libs/1_58_0/libs/python/doc/index.html)
* [Boost.python on python wiki](https://wiki.python.org/moin/boost.python/HowTo)
* [Workshop at scipy 2014 on boost-python](https://github.com/jcfr/scipy2014_boost_python_workshop_student_material)
* [Neet mingling of boost::python in an IPython notebook](https://github.com/abingham/boost_python_tutorial)


The `graph_tool` library is a very good example of higly efficient, complete and complex boost based python library: 
* [graph-tool source code](https://git.skewed.de/count0/graph-tool/tree/master)

### CGAL
* [CGAL documentation](http://doc.cgal.org)
* [CGAL Polyhedron](http://doc.cgal.org/latest/Polyhedron/index.html)
* [CGAL LinearCellComplex](http://doc.cgal.org/latest/Linear_cell_complex/group__PkgLinearCellComplex.html)
* [CGAL CombinatorialMap](http://doc.cgal.org/latest/Combinatorial_map/index.html#ChapterCombinatorialMap)



## Models we want to simulate

The first model that will be implemented is the one described in
Monier et al. [monier2015apico]. It is an example of a vertex model,
where the interactions are only evaluated on the apical surface sheet
of the epithelium. The second class of models are still at an
stage. They implement a description of the tissue's rheology, within a
dissipation function formalism.

![Examples of possible models](../../doc/illus/two_models.png)

## Class diagram

The graphic below details the various objects modeled here. Each of those are implemented as an abstract class. If one considers the model in Monier et al. [monier2015apico], the apical sheet is consituted of vertices and edges, representing apical junctions. This geometry can be generalized to model 3D columnar cells constituted of apical, basal and equatorial meshes, linked by sagital edges. Depending on the model, a subset of those junctions will be used. Each cell interacts with its neighbour through its junctions. An epithelium is then a graph of interacting cells, with two cells neighbouring cells sharing a junction.

There are two interesting libraries to deal with the objects we whish to consider. The first one, [`CGAL::Polyhedron`](http://doc.cgal.org/latest/Polyhedron/index.html) can only represent surfaces, and can't be used for the general 3D tissue. The second one, [`CGAL::LinearCellComplex`](http://doc.cgal.org/latest/Linear_cell_complex/group__PkgLinearCellComplex.html) is suited for all our models. The `Polyhedron` model is simpler though, and there's a [function](http://doc.cgal.org/latest/Linear_cell_complex/group__PkgLinearCellComplexConstructions.html#ga9239e988c6d5e674d64438d47b95a548) that can create `LCC` objects form `Polyhedron` instances. In order to ease the porting of the `leg-joint` simulations to the new architecture, we'll first develop this model within the `Polyhedron` framework, and then generalized with the LCC.

### `Polyhedron` class diagramm

Here is a representation of the data structure defined by the `CGAL::Polyhedron` data structure:

![Definition of a polyedron_3 data structure from CGAL's doc](http://doc.cgal.org/latest/Polyhedron/halfedge_small.png)

When modeling an epithelium sheet (apical, basal or sagital), there is a direct equivalence between:

* Junction vertex and Vertex 
* Junction edge and Edge (represented by two halfedges)
* Facet and cell

![Class diagram for polyhedron](../../doc/illus/class_diagramm_poly.png)




### `LCC` class diagramm


All the physical objects are represented by CGAL Linear Cell Complexes - in short LCC, that are specialized versions of CGAL Combinatorial Maps. Vertices are 0-cells (associated to a Point object in the LCC), edges 1-cells, faces are 2-cells and volumes are 3-cells [1]. With this concept, individual cells as well as the whole epithelium are represented by a LCC. As can be seen by browsing the above linked documentations, this very general framework provides efficient ways to represent the tissue and common events such as cell division (using lcc.insert_point_in_cell). For a given model, only subsets of these objects might be needed, but we'll try to keep the various definitions as generic as possible, for example by defining cell division irrespective of the detailed geometry of the cell, through template meta-programming


![Class diagram](../../doc/illus/class_diagram.png)
_Not meant to be ULM compliant_


Note that we might need to further sub-class CGAL's LinearCellComplex to take into account the possibility of curved edges and faces in the vertex rheology models.

### LinearCellComplex in CGAL

In CGAL, the LinearCellComplex defines linear oriented nD objects. In 2D, this representation is equivalent to the [HalfEdge data structure](http://doc.cgal.org/latest/HalfedgeDS/index.html#chapterHalfedgeDS) and could be implemented as [Polygon_3](http://doc.cgal.org/latest/Polyhedron/group__PkgPolyhedron.html). The advantage of LCC over halfedges is it's extensibility to 3 dimentions and dimention agnostic aspects.

![Schematic of a CGAL LCC](http://doc.cgal.org/latest/Linear_cell_complex/lcc-examples-2d-3d.png)

As a LCC is a CombinatorialMap, and as such is **edge centered**. A combinatorial map is a set of **darts** linking all its elements. The concept of darts and $\beta_i$ pointers is discussed at length in CGAL documentation. We will mainly use them to define custom iterators over the various elements of our tissues. 

#### Over libraries of interest in CGAL

* [Boost property maps](http://doc.cgal.org/latest/Property_map/index.html)
* [I/O](http://doc.cgal.org/latest/Stream_support/index.html#Chapter_IO_Streams). We might prefer to
  defer I/O to python, though.


## Definitions of the building blocks

The sketch below details the overall tissue organisation, and the different building blocks a tissue is composed of.

![Objects architecture](../../doc/illus/objects_architecture.png)

Note that the vertex called "cell vertex" and the "cell to junction" edges are an oversegmentation of the tissue, that might be usefull to compute geometrical properties or for visualisation (as a mesh). Actual cells can be represented by a single n-cell LCC, or a combination of n-cells LCC, where n is the tissue dimention (n is 2 for a sheet, 3 for a volume).


###  Notations


| Object   | index notation  | iterator notation  |
| -------- | ---------------- | ------------------- |
| Junction vertex | $jv^a_i$ | $i = {0, \cdots, N_{ajv}}$ |
| Bulk vertex | $bv^a_\alpha$  | $\alpha = {0, \cdots, N_{c}}$  |
| Junction edge | $je^a_{ij}$ | $i,j / j \neq i, (i, j) \in C(i)$ |

Here $C(i)$ is the set of junction edges going out of vertex $i$. The superscript $a$ stends for apical.

### Project file structure

* [tyssue project](../../)
    - [notebooks](../)
        * sympy _Models implementation in sympy_
        * core-architecture _Technical documentation of the project implementation_
        * simulations _Documentation of the python frontend_
    - [illus](../../illus) _Sketches and illustrations_
    - [src](../../src) _The source code_
        * cpp-tyssue _C++ files_
        * tyssue _Python bindings_
        

## Python implementation of the class structure

For clarity, and avoiding too verbose pseudo-code, we don't show the implementation of the bulk oversegmentation LCCs.

In [1]:
import tyssue


In [2]:
eptm = tyssue.core.Epithelium()

In [1]:

class JunctionVertex():
    
    def __init__(self, ):
        
        
        
class JunctionEdge(LinearCellComplex):
    
    def __init__(self):

class Facet(LinearCellComplex):
    
    def __init__(self):
        LinearCellComplex.__init__(self, 2)

class Volume(LinearCellComplex):
    
    def __init__(self):
        LinearCellComplex.__init__(self, 3)

        
class Cell(LinearCellComplex):
    
    def __init__(self, dim):
        LinearCellComplex.__init__(self, dim)


    @property
    def j_edges(self):
        '''
        Iterate over the junction edges
        '''
        for je in self._jnct_edges:
            yield je

        
    @property
    def faces(self):
        '''
        Iterate over the faces
        '''
        for face in self._faces:
            yield face
        
